In [2]:
from ape import Contract, chain, networks,accounts,project,convert,api
from web3 import Web3, AsyncWeb3
import json


w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:7545'))
w3.is_connected()

w3.eth.default_account = w3.eth.accounts[0]
print("default account balance:",w3.eth.default_account, w3.eth.get_balance(w3.eth.default_account))

default account balance: 0x33473533dBFb294d54CCeDD8fc4301854DfD1948 100000000000000000000


In [3]:
project.CF

<CF>

In [4]:
with networks.parse_network_choice("ethereum:local:ganache") as provider:
    js = project.CF.contract_type.dict()
    # ["abi"]
    print(js)
    print(list(js.keys()))


ExampleContract = w3.eth.contract(abi=js["abi"], bytecode=js["deploymentBytecode"]["bytecode"])
tx_hash = ExampleContract.constructor(w3.to_wei(3,"ether")).transact()
tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)


contract = w3.eth.contract(address=tx_receipt.contractAddress, abi=js["abi"])

contract.functions.isOwner().call()

INFO: Starting 'Ganache' process.
{'contractName': 'CF', 'sourceId': 'crowdfunding.sol', 'deploymentBytecode': {'bytecode': '0x608060405234801561001057600080fd5b50604051610d19380380610d1983398101604081905261002f91610054565b600080546001600160a01b031916331790556001556004805461ffff1916905561006d565b60006020828403121561006657600080fd5b5051919050565b610c9d8061007c6000396000f3fe6080604052600436106100c25760003560e01c8063a4475ce41161007f578063d424f62811610059578063d424f6281461020c578063d7bb99ba14610226578063fc0e97391461022e578063fe47a8a71461024357600080fd5b8063a4475ce414610199578063af157c19146101b9578063c6221878146101dd57600080fd5b806324600fc3146100c7578063253da8bd146100de5780632636b945146101025780633cb5d1001461011857806356c7f4611461015d5780638f32d59b14610172575b600080fd5b3480156100d357600080fd5b506100dc610259565b005b3480156100ea57600080fd5b506003545b6040519081526020015b60405180910390f35b34801561010e57600080fd5b506100ef60015481565b34801561012457600080fd5b50610138610133366004610a7e565b61033f565

'0x33473533dBFb294d54CCeDD8fc4301854DfD1948'

In [5]:
# Get deployment bytecode
# deploy_bytecode = w3.eth.contract(abi=[]).constructor().buildTransaction({'data': contract_bytecode})['data']

# Estimate gas 
# gas_estimate = w3.eth.estimate_gas({
#   'from': w3.eth.accounts[0], 
#   'data': js["deploymentBytecode"]["bytecode"]
# })
# w3.eth.estimate_gas(tx_receipt)
ExampleContract.constructor(1).estimate_gas()

798558

In [4]:
import pprint

amount = 1
# account = "0xDc541B8b0e326bdf734744f497713A7cb2255b0F"
# private_key = "0x0aa622a36df1c7db6007e8d03f7b42fe1d7224cfc1cd1fc6cf7351003c70a668"
# w3.eth.default_account = account

# nonce = w3.eth.get_transaction_count(w3.eth.accounts[1])

contributors = [w3.eth.accounts[1],w3.eth.accounts[2]]

for contributor in contributors:

    tx = {
            "chainId": w3.eth.chain_id,  # Replace with the chain ID of your network
            "from": contributor,
            # "to": tx_receipt.contractAddress,
            "value": w3.to_wei(amount, "ether"),
            # "gas": gas,
            # "gasPrice": gasPrice,
            "gas": 200000,  # Specify the gas limit
            "gasPrice": w3.to_wei("20", "gwei"),  # Specify the gas price
            # "nonce": nonce,
        }

    tx_hash = contract.functions.contribute().transact(tx)
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    pprint.pprint(dict(tx_receipt))

# transaction = contract.functions.contribute().build_transaction(tx)

# signed_transaction = w3.eth.account.sign_transaction(transaction, private_key)
# transaction_hash = w3.eth.send_raw_transaction(signed_transaction.rawTransaction)

# # Render the result template with the transaction hash
# transaction_hash=transaction_hash.hex()
# transaction_hash

{'blockHash': HexBytes('0x57f2960b5aebe9f0d992fdc5fe5f0dbfdd41df1e5bf96834049368ca42c8395c'),
 'blockNumber': 280,
 'contractAddress': None,
 'cumulativeGasUsed': 137788,
 'effectiveGasPrice': 20000000000,
 'from': '0xC15c7c758Ff39775779917370BED67b420DB7cdf',
 'gasUsed': 137788,
 'logs': [AttributeDict({'address': '0x20b1Ee7d3Bf87228f6472a96958121Db49Da8aE1', 'blockHash': HexBytes('0x57f2960b5aebe9f0d992fdc5fe5f0dbfdd41df1e5bf96834049368ca42c8395c'), 'blockNumber': 280, 'data': HexBytes('0x000000000000000000000000c15c7c758ff39775779917370bed67b420db7cdf0000000000000000000000000000000000000000000000000de0b6b3a7640000'), 'logIndex': 0, 'removed': False, 'topics': [HexBytes('0x23bac35767dec08bf219a302474bbc6687dfaaf3776a62d45bb7811de040e26d')], 'transactionHash': HexBytes('0x80bd7b750b750d4c2ef02693c352f874158556ac52419ee31e632472b2ab38db'), 'transactionIndex': 0})],
 'logsBloom': HexBytes('0x000000040000000000000000000000000800000000000000000000000000000000000000000000000000000000000000

In [5]:
try:
    tx_hash=contract.functions.completeProject().transact()
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    print(tx_receipt)
except Exception as e:
    print(e)

execution reverted: VM Exception while processing transaction: revert Funding goal has not been reached.


In [6]:
# Example: Check crowdfunding status
def check_crowdfunding_status():
    project_owner = contract.functions.projectOwner().call()
    goal_amount = contract.functions.goalAmount().call()
    total_funding = contract.functions.totalFunding().call()
    funding_goal_reached = contract.functions.fundingGoalReached().call()
    project_completed = contract.functions.projectCompleted().call()

    print("contractAddress:", contract.address)
    print(f"Project Owner: {project_owner}")
    print(f"Default Account: {w3.eth.default_account}")
    print(f"Goal Amount: {goal_amount} Wei")
    print(f"Goal Amount: {w3.from_wei(goal_amount,'ether')} Ether")
    print(f"Total Funding: {total_funding} Wei")
    print(f"Total Funding: {w3.from_wei(total_funding,'ether')} Ether")

    print(f"Funding Goal Reached: {funding_goal_reached}")
    print(f"Project Completed: {project_completed}")
    print("getContributorCount: ", contract.functions.getContributorCount().call())
    # print("getContributorAtIndex:",contract.functions.getContributors().call())
    (
        contributor_addresses,
        contribution_amounts,
        contribution_datetime,
    ) = contract.functions.getContributors().call()
    print("contributors:::::")
    for i in range(len(contributor_addresses)):
        # contributors_dict[contributor_addresses[i].hex()] = contribution_amounts[i]
        print(
            contributor_addresses[i], contribution_amounts[i], contribution_datetime[i]
        )


check_crowdfunding_status()

contractAddress: 0x20b1Ee7d3Bf87228f6472a96958121Db49Da8aE1
Project Owner: 0xDc541B8b0e326bdf734744f497713A7cb2255b0F
Default Account: 0xDc541B8b0e326bdf734744f497713A7cb2255b0F
Goal Amount: 3000000000000000000 Wei
Goal Amount: 3 Ether
Total Funding: 2000000000000000000 Wei
Total Funding: 2 Ether
Funding Goal Reached: False
Project Completed: False
getContributorCount:  2
contributors:::::
0xC15c7c758Ff39775779917370BED67b420DB7cdf 1000000000000000000 1699773381
0x3f31f6284EAC0d0C3bf86E1C1E9649D8FDDf59D4 1000000000000000000 1699773381


In [7]:
try:
    tx_hash=contract.functions.refundContributors().transact()
    tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
    pprint.pprint(dict(tx_receipt))
except Exception as e:
    print(e)

{'blockHash': HexBytes('0xf510c83fd250124d28e3a97c03ef47d37a35e510a59cbc3cd6464b8d1bb03725'),
 'blockNumber': 282,
 'contractAddress': None,
 'cumulativeGasUsed': 61375,
 'effectiveGasPrice': 10,
 'from': '0xDc541B8b0e326bdf734744f497713A7cb2255b0F',
 'gasUsed': 61375,
 'logs': [AttributeDict({'address': '0x20b1Ee7d3Bf87228f6472a96958121Db49Da8aE1', 'blockHash': HexBytes('0xf510c83fd250124d28e3a97c03ef47d37a35e510a59cbc3cd6464b8d1bb03725'), 'blockNumber': 282, 'data': HexBytes('0x000000000000000000000000c15c7c758ff39775779917370bed67b420db7cdf0000000000000000000000000000000000000000000000000de0b6b3a76400000000000000000000000000000000000000000000000000000000000065507bc5'), 'logIndex': 0, 'removed': False, 'topics': [HexBytes('0x0754663d79ddf47e96611d36d595a38ce0918960aa5965913d75c482d0477ed6')], 'transactionHash': HexBytes('0x940d50ed845c7ce1c29b69be1c7e4f7feb78fd115891f12756e62a32c9d18de7'), 'transactionIndex': 0}),
          AttributeDict({'address': '0x20b1Ee7d3Bf87228f6472a96958121D

In [8]:
# tx_hash=contract.functions.withdrawFunds().transact()
# tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
# tx_receipt

In [9]:
# tx_hash=contract.functions.contributions().call()
# tx_receipt = w3.eth.wait_for_transaction_receipt(tx_hash)
# tx_receipt

In [10]:
tx_receipt["logs"][0]

AttributeDict({'address': '0x20b1Ee7d3Bf87228f6472a96958121Db49Da8aE1',
 'blockHash': HexBytes('0xf510c83fd250124d28e3a97c03ef47d37a35e510a59cbc3cd6464b8d1bb03725'),
 'blockNumber': 282,
 'data': HexBytes('0x000000000000000000000000c15c7c758ff39775779917370bed67b420db7cdf0000000000000000000000000000000000000000000000000de0b6b3a76400000000000000000000000000000000000000000000000000000000000065507bc5'),
 'logIndex': 0,
 'removed': False,
 'topics': [HexBytes('0x0754663d79ddf47e96611d36d595a38ce0918960aa5965913d75c482d0477ed6')],
 'transactionHash': HexBytes('0x940d50ed845c7ce1c29b69be1c7e4f7feb78fd115891f12756e62a32c9d18de7'),
 'transactionIndex': 0})

In [11]:
tx_receipt["logs"][0]["transactionHash"].hex()

'0x940d50ed845c7ce1c29b69be1c7e4f7feb78fd115891f12756e62a32c9d18de7'

In [12]:
contract.events.RefundedContributor().get_logs()

(AttributeDict({'args': AttributeDict({'contributor': '0xC15c7c758Ff39775779917370BED67b420DB7cdf',
   'amount': 1000000000000000000,
   'timestamp': 1699773381}),
  'event': 'RefundedContributor',
  'logIndex': 0,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x940d50ed845c7ce1c29b69be1c7e4f7feb78fd115891f12756e62a32c9d18de7'),
  'address': '0x20b1Ee7d3Bf87228f6472a96958121Db49Da8aE1',
  'blockHash': HexBytes('0xf510c83fd250124d28e3a97c03ef47d37a35e510a59cbc3cd6464b8d1bb03725'),
  'blockNumber': 282}),
 AttributeDict({'args': AttributeDict({'contributor': '0x3f31f6284EAC0d0C3bf86E1C1E9649D8FDDf59D4',
   'amount': 1000000000000000000,
   'timestamp': 1699773381}),
  'event': 'RefundedContributor',
  'logIndex': 1,
  'transactionIndex': 0,
  'transactionHash': HexBytes('0x940d50ed845c7ce1c29b69be1c7e4f7feb78fd115891f12756e62a32c9d18de7'),
  'address': '0x20b1Ee7d3Bf87228f6472a96958121Db49Da8aE1',
  'blockHash': HexBytes('0xf510c83fd250124d28e3a97c03ef47d37a35e510a59cbc3cd6464b